### Splitting the Name column into:
- make (e.g., Toyota)
- model (e.g., Premio)
- year (e.g., 2015)
- engine_info (e.g., 1.8 FWD)
- color (e.g., Black)

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [2]:
df = pd.read_csv("../data/raw/carpredict_data.csv")
df.head(10)

,price,transmission,condition,mileage,Name
0,2380000,Automatic,Foreign Used,85644.0,Subaru Legacy 2015 Silver
1,1050000,Automatic,Foreign Used,98560.0,Toyota Sienta 2014 1.5 FWD Gray
2,7000000,Automatic,Foreign Used,56918.0,Toyota Land Cruiser Prado 2015 2.8 D-4d White
3,1950000,Automatic,Foreign Used,66000.0,Toyota Premio 2015 1.8 FWD Black
4,920000,Automatic,Foreign Used,42308.0,Toyota Vitz 2015 1.3 AWD 5dr Silver
5,1650000,Automatic,Kenyan Used,120589.0,Toyota Harrier 2009 2.4 White
6,6000000,Automatic,Foreign Used,20000.0,Toyota Land Cruiser Prado 2014 2.7 VVT-i Pearl
7,2840000,Automatic,Kenyan Used,123000.0,BMW X3 2011 xDrive28i Black
8,1300000,Automatic,Foreign Used,48565.0,Subaru Impreza 2015 White
9,930000,Automatic,Foreign Used,65300.0,Toyota Spade 2014 1.5 Yellow


In [3]:
print("=== STEP 1: EXTRACTING MAKE ===")
print("Sample Name values:")
for i in range(10):
    print(f"{i}: {df['Name'].iloc[i]}")

print("\n=== APPROACH: Extract First Word as Make ===")

def extract_make(name):
    """
    Extract the make (manufacturer) from the Name column.
    Make is  the first word in the name.
    """
    if pd.isna(name):
        return 'Unknown'
    
    # Split by space and take the first word
    parts = str(name).strip().split()
    if len(parts) == 0:
        return 'Unknown'
    
    # Clean the make (remove any special characters, convert to title case)
    make = parts[0].strip()
    make = ''.join(char for char in make if char.isalnum())  # Remove special chars
    return make.title()  # Convert to title case (Toyota, not TOYOTA or toyota)

# Apply the function
df['make'] = df['Name'].apply(extract_make)

=== STEP 1: EXTRACTING MAKE ===
Sample Name values:
0: Subaru Legacy 2015 Silver
1: Toyota Sienta 2014 1.5 FWD Gray
2: Toyota Land Cruiser Prado 2015 2.8 D-4d White
3: Toyota Premio 2015 1.8 FWD Black
4: Toyota Vitz 2015 1.3 AWD 5dr Silver
5: Toyota Harrier 2009 2.4 White
6: Toyota Land Cruiser Prado 2014 2.7 VVT-i Pearl
7: BMW X3 2011 xDrive28i Black
8: Subaru Impreza 2015 White
9: Toyota Spade 2014 1.5 Yellow

=== APPROACH: Extract First Word as Make ===


In [4]:
print("=== STEP 2: EXTRACTING MODEL ===")
print("Strategy: Extract model after removing make and before year/specs")

# First, let's analyze the structure after removing the make
print("=== ANALYZING NAME STRUCTURE ===")
print("Sample analysis of what remains after make:")

def analyze_remaining_parts(name, make):
    """Analyze what remains after removing the make"""
    if pd.isna(name) or pd.isna(make):
        return "N/A"
    
    name_parts = str(name).strip().split()
    make_parts = str(make).strip().split()
    
    # Remove make from the beginning
    remaining_parts = name_parts[len(make_parts):]
    return ' '.join(remaining_parts)

# Show sample analysis
print("Name -> Make -> Remaining parts:")
for i in range(10):
    name = df['Name'].iloc[i]
    make = df['make'].iloc[i]
    remaining = analyze_remaining_parts(name, make)
    print(f"{name:<45} -> {make:<12} -> {remaining}")

print("\n=== MODEL EXTRACTION FUNCTION ===")

def extract_model(name, make):
    """
    Extract the model from the Name column after removing the make.
    The model typically comes after the make and before the year.
    """
    if pd.isna(name) or pd.isna(make):
        return 'Unknown'
    
    # Convert to string and split
    name_parts = str(name).strip().split()
    make_lower = str(make).lower()
    
    # Remove make parts from the beginning
    # Handle cases where make might be compound (like "Land Rover")
    remaining_parts = []
    make_found = False
    
    for i, part in enumerate(name_parts):
        if not make_found and part.lower().startswith(make_lower.lower()):
            make_found = True
            continue
        elif make_found:
            remaining_parts = name_parts[i:]
            break
    
    if not remaining_parts:
        # If we couldn't find make, assume first part is make, rest is model
        remaining_parts = name_parts[1:] if len(name_parts) > 1 else []
    
    if not remaining_parts:
        return 'Unknown'
    
    # Find where the year starts (4-digit number starting with 19 or 20)
    year_index = None
    for i, part in enumerate(remaining_parts):
        if re.match(r'^(19|20)\d{2}$', part):
            year_index = i
            break
    
    # Extract model parts (everything before the year)
    if year_index is not None:
        model_parts = remaining_parts[:year_index]
    else:
        # If no year found, we need to be more careful
        # Look for common patterns that indicate end of model name
        model_parts = []
        for part in remaining_parts:
            # Stop if we hit common engine/spec patterns
            if re.match(r'^\d+\.\d+$', part):  # Engine size like "2.0", "1.8"
                break
            elif part.upper() in ['FWD', 'AWD', '4WD', 'RWD', 'CVT', 'MT', 'AT']:
                break
            elif re.match(r'^[A-Z]{2,4}$', part) and len(part) <= 4:  # Short acronyms
                break
            elif part.lower() in ['black', 'white', 'silver', 'gray', 'grey', 'blue', 'red', 'green']:
                break
            else:
                model_parts.append(part)
    
    if not model_parts:
        return 'Unknown'
    
    # Join model parts and clean
    model = ' '.join(model_parts).strip()
    
    # Clean up the model name
    model = re.sub(r'[^\w\s-]', '', model)  # Remove special chars except hyphens
    model = ' '.join(model.split())  # Normalize whitespace
    
    return model.title() if model else 'Unknown'

# Apply the function
df['model'] = df.apply(lambda row: extract_model(row['Name'], row['make']), axis=1)

=== STEP 2: EXTRACTING MODEL ===
Strategy: Extract model after removing make and before year/specs
=== ANALYZING NAME STRUCTURE ===
Sample analysis of what remains after make:
Name -> Make -> Remaining parts:
Subaru Legacy 2015 Silver                     -> Subaru       -> Legacy 2015 Silver
Toyota Sienta 2014 1.5 FWD Gray               -> Toyota       -> Sienta 2014 1.5 FWD Gray
Toyota Land Cruiser Prado 2015 2.8 D-4d White -> Toyota       -> Land Cruiser Prado 2015 2.8 D-4d White
Toyota Premio 2015 1.8 FWD Black              -> Toyota       -> Premio 2015 1.8 FWD Black
Toyota Vitz 2015 1.3 AWD 5dr Silver           -> Toyota       -> Vitz 2015 1.3 AWD 5dr Silver
Toyota Harrier 2009 2.4 White                 -> Toyota       -> Harrier 2009 2.4 White
Toyota Land Cruiser Prado 2014 2.7 VVT-i Pearl -> Toyota       -> Land Cruiser Prado 2014 2.7 VVT-i Pearl
BMW X3 2011 xDrive28i Black                   -> Bmw          -> X3 2011 xDrive28i Black
Subaru Impreza 2015 White                    

In [5]:
print("=== STEP 3: EXTRACTING YEAR AND CALCULATING AGE ===")

# Set the current year for age calculation
CURRENT_YEAR = 2025  # Adjust this if needed

print(f"Using {CURRENT_YEAR} as the current year for age calculation")

print("\n=== YEAR EXTRACTION FUNCTION ===")

def extract_year(name):
    """
    Extract the year from the Name column.
    Look for 4-digit numbers starting with 19 or 20.
    """
    if pd.isna(name):
        return None
    
    # Convert to string
    name_str = str(name)
    
    # Find all 4-digit years (19xx or 20xx)
    year_matches = re.findall(r'\b(19[0-9]{2}|20[0-9]{2})\b', name_str)
    
    if year_matches:
        # Convert to integers and take the first valid year
        years = [int(year) for year in year_matches]
        
        # Filter for reasonable car years (1900-2030)
        valid_years = [year for year in years if 1900 <= year <= 2030]
        
        if valid_years:
            # If multiple years found, take the first one (most likely to be manufacture year)
            return valid_years[0]
    
    return None

# Apply year extraction
df['year'] = df['Name'].apply(extract_year)

print("\n=== CALCULATE CAR AGE ===")

def calculate_car_age(year, current_year=CURRENT_YEAR):
    """Calculate car age based on manufacture year"""
    if pd.isna(year):
        return None
    return current_year - int(year)

# Calculate car age
df['car_age'] = df['year'].apply(lambda x: calculate_car_age(x))

=== STEP 3: EXTRACTING YEAR AND CALCULATING AGE ===
Using 2025 as the current year for age calculation

=== YEAR EXTRACTION FUNCTION ===

=== CALCULATE CAR AGE ===


In [6]:
print("=== STEP 4: EXTRACTING ENGINE INFO ===")
print("Strategy: Extract everything between year and color")

# Common color names to help identify where engine info ends
COMMON_COLORS = {
    "Black",
    "White",
    "Silver",
    "Gray",
    "Grey",
    "Blue",
    "Red",
    "Green",
    "Brown",
    "Gold",
    "Yellow",
    "Orange",
    "Purple",
    "Beige",
    "Tan",
    "Pink",
    "Burgundy",

    # Color variations
    "Pearl",
    "Metallic",
    "Pearl White",
    "Pearl Black",
    "Dark Blue",
    "Light Blue",
    "Dark Gray",
    "Light Gray",
    "Dark Grey",   # duplicate spelled variation keeps same title
    "Light Grey",
    "Maroon",
    "Navy",
    "Cream",
    "Champagne",

    # Specific car color names
    "Gunmetal",
    "Charcoal",
    "Midnight",
    "Ivory",
    "Bronze",
    "Copper"
}

print("\n=== ANALYZING NAME STRUCTURE ===")
print("Sample analysis of parts after year:")

def analyze_post_year_parts(name, year):
    """Analyze what comes after the year in the name"""
    if pd.isna(name) or pd.isna(year):
        return "N/A"
    
    name_str = str(name)
    year_str = str(int(year))
    
    # Find where the year appears in the name
    year_match = re.search(rf'\b{year_str}\b', name_str)
    if year_match:
        # Get everything after the year
        post_year = name_str[year_match.end():].strip()
        return post_year
    
    return "Year not found"

# Show sample analysis
print("Name -> Year -> Parts after year:")
sample_with_year = df[df['year'].notna()].head(10)
for idx, row in sample_with_year.iterrows():
    name = row['Name']
    year = row['year']
    post_year = analyze_post_year_parts(name, year)
    print(f"{name:<45} -> {year} -> '{post_year}'")

print("\n=== ENGINE INFO EXTRACTION FUNCTION ===")

def extract_engine_info(name, year):
    """
    Extract engine info from the Name column.
    Engine info is everything between the year and the color.
    """
    if pd.isna(name) or pd.isna(year):
        return 'Unknown'
    
    name_str = str(name).strip()
    year_str = str(int(year))
    
    # Find the year position
    year_match = re.search(rf'\b{year_str}\b', name_str)
    if not year_match:
        return 'Unknown'
    
    # Get everything after the year
    after_year = name_str[year_match.end():].strip()
    
    if not after_year:
        return 'Unknown'
    
    # Split into parts
    parts = after_year.split()
    
    if not parts:
        return 'Unknown'
    
    # Find where the color starts (from the end, working backwards)
    engine_parts = []
    color_found = False
    
    # Check from the last part backwards to find color
    for i in range(len(parts) - 1, -1, -1):
        part_lower = parts[i].lower()
        
        # If this part is a color, everything from here to the end is color
        if part_lower in COMMON_COLORS:
            # Everything before this index is engine info
            engine_parts = parts[:i]
            color_found = True
            break
    
    # If no color found, assume last part is color and rest is engine info
    if not color_found and len(parts) > 1:
        # Check if the last part looks like a color (common pattern)
        last_part = parts[-1].lower()
        if len(last_part) > 2 and last_part.isalpha():  # Likely a color word
            engine_parts = parts[:-1]
        else:
            # If last part doesn't look like color, include everything
            engine_parts = parts
    elif not color_found:
        # Only one part after year, could be engine info or color
        # If it's clearly not a color, treat as engine info
        single_part = parts[0].lower()
        if single_part not in COMMON_COLORS and not single_part.isalpha():
            engine_parts = parts
        else:
            engine_parts = []  # Likely just a color
    
    # Clean and join engine parts
    if engine_parts:
        engine_info = ' '.join(engine_parts).strip()
        # Clean up the engine info
        engine_info = re.sub(r'\s+', ' ', engine_info)  # Normalize whitespace
        return engine_info if engine_info else 'Unknown'
    
    return 'Unknown'

# Apply the function to rows with year data
df['engine_info'] = df.apply(lambda row: extract_engine_info(row['Name'], row['year']), axis=1)

=== STEP 4: EXTRACTING ENGINE INFO ===
Strategy: Extract everything between year and color

=== ANALYZING NAME STRUCTURE ===
Sample analysis of parts after year:
Name -> Year -> Parts after year:
Subaru Legacy 2015 Silver                     -> 2015 -> 'Silver'
Toyota Sienta 2014 1.5 FWD Gray               -> 2014 -> '1.5 FWD Gray'
Toyota Land Cruiser Prado 2015 2.8 D-4d White -> 2015 -> '2.8 D-4d White'
Toyota Premio 2015 1.8 FWD Black              -> 2015 -> '1.8 FWD Black'
Toyota Vitz 2015 1.3 AWD 5dr Silver           -> 2015 -> '1.3 AWD 5dr Silver'
Toyota Harrier 2009 2.4 White                 -> 2009 -> '2.4 White'
Toyota Land Cruiser Prado 2014 2.7 VVT-i Pearl -> 2014 -> '2.7 VVT-i Pearl'
BMW X3 2011 xDrive28i Black                   -> 2011 -> 'xDrive28i Black'
Subaru Impreza 2015 White                     -> 2015 -> 'White'
Toyota Spade 2014 1.5 Yellow                  -> 2014 -> '1.5 Yellow'

=== ENGINE INFO EXTRACTION FUNCTION ===


In [7]:
print("=== STEP 5: EXTRACTING COLOR (FINAL PARSING STEP) ===")
print("Strategy: Extract color from the end of the name after all other components")

print("\n=== ANALYZING COLOR PATTERNS ===")
print("Let's see what's left at the end of names after extracting other components:")

# Show sample of what should be color at the end
print("Sample analysis of remaining parts (should be colors):")
for i in range(15):
    name = df['Name'].iloc[i]
    make = df.get('make', pd.Series([None]*len(df))).iloc[i]
    model = df.get('model', pd.Series([None]*len(df))).iloc[i]
    year = df.get('year', pd.Series([None]*len(df))).iloc[i]
    engine = df.get('engine_info', pd.Series([None]*len(df))).iloc[i]
    
    print(f"Full name: {name}")
    
    # Try to identify what should be the color by elimination
    remaining = name
    if make and str(make) != 'Unknown':
        remaining = remaining.replace(str(make), '', 1).strip()
    if model and str(model) != 'Unknown':
        remaining = remaining.replace(str(model), '', 1).strip()
    if year and not pd.isna(year):
        remaining = remaining.replace(str(int(year)), '').strip()
    if engine and str(engine) != 'Unknown':
        remaining = remaining.replace(str(engine), '').strip()
    
    print(f"  After removing components: '{remaining}'")
    print()

print("=== COLOR EXTRACTION FUNCTION ===")

# Define comprehensive color dictionary
COLORS = {
    # Basic colors
    'black': 'Black',
    'white': 'White', 
    'silver': 'Silver',
    'gray': 'Gray',
    'grey': 'Gray',  # Standardize to Gray
    'blue': 'Blue',
    'red': 'Red',
    'green': 'Green',
    'brown': 'Brown',
    'gold': 'Gold',
    'yellow': 'Yellow',
    'orange': 'Orange',
    'purple': 'Purple',
    'beige': 'Beige',
    'tan': 'Tan',
    'pink': 'Pink',
    'burgandy': 'Burgandy',
    
    # Color variations
    'pearl': 'Pearl',
    'metallic': 'Metallic',
    'pearl white': 'Pearl White',
    'pearl black': 'Pearl Black',
    'dark blue': 'Dark Blue',
    'light blue': 'Light Blue',
    'dark gray': 'Dark Gray',
    'light gray': 'Light Gray',
    'dark grey': 'Dark Gray',
    'light grey': 'Light Gray',
    'maroon': 'Maroon',
    'burgundy': 'Burgundy',
    'navy': 'Navy',
    'cream': 'Cream',
    'champagne': 'Champagne',
    
    # Specific car color names
    'gunmetal': 'Gunmetal',
    'charcoal': 'Charcoal',
    'midnight': 'Midnight',
    'ivory': 'Ivory',
    'bronze': 'Bronze',
    'copper': 'Copper'
}

def extract_color(name, make=None, model=None, year=None, engine_info=None):
    """
    Extract color from the Name column.
    Color typically appears at the end of the name string.
    """
    if pd.isna(name):
        return 'Unknown'
    
    name_str = str(name).strip().lower()
    
    # Method 1: Look for colors at the end of the string
    words = name_str.split()
    
    # Check last few words for color matches
    for i in range(min(3, len(words))):  # Check last 1-3 words
        # Single word color check
        last_word = words[-(i+1)]
        if last_word in COLORS:
            return COLORS[last_word]
        
        # Two-word color check (like "pearl white", "dark blue")
        if i < len(words) - 1:
            two_words = f"{words[-(i+2)]} {words[-(i+1)]}"
            if two_words in COLORS:
                return COLORS[two_words]

# Apply color extraction
df['color'] = df.apply(lambda row: extract_color(
    row['Name'],
    row.get('make'),
    row.get('model'), 
    row.get('year'),
    row.get('engine_info')
), axis=1)

=== STEP 5: EXTRACTING COLOR (FINAL PARSING STEP) ===
Strategy: Extract color from the end of the name after all other components

=== ANALYZING COLOR PATTERNS ===
Let's see what's left at the end of names after extracting other components:
Sample analysis of remaining parts (should be colors):
Full name: Subaru Legacy 2015 Silver
  After removing components: 'Silver'

Full name: Toyota Sienta 2014 1.5 FWD Gray
  After removing components: 'Gray'

Full name: Toyota Land Cruiser Prado 2015 2.8 D-4d White
  After removing components: 'White'

Full name: Toyota Premio 2015 1.8 FWD Black
  After removing components: 'Black'

Full name: Toyota Vitz 2015 1.3 AWD 5dr Silver
  After removing components: 'Silver'

Full name: Toyota Harrier 2009 2.4 White
  After removing components: 'White'

Full name: Toyota Land Cruiser Prado 2014 2.7 VVT-i Pearl
  After removing components: 'Pearl'

Full name: BMW X3 2011 xDrive28i Black
  After removing components: 'BMW    Black'

Full name: Subaru Impreza 

In [14]:
df = df.drop('Name', axis=1)

In [15]:
df.sample(15)

,price,transmission,condition,mileage,make,model,year,car_age,engine_info,color
7412,11000000,Automatic,Foreign Used,25000.0,Mercedesbenz,G-Class,2015,10,G 350 BlueTEC,Black
8262,1050000,Automatic,Kenyan Used,156000.0,Mitsubishi,Outlander,2008,17,2.4i GLS Automatic 4x4,Gray
187,9000000,Automatic,Foreign Used,53000.0,Toyota,Land Cruiser,2015,10,4.6 V8 ZX,White
2883,2799999,Automatic,Foreign Used,NaN,Audi,A5,2014,11,Unknown,Black
7622,970000,Automatic,Brand New,2100.0,New,Proton Saga,2020,5,Premium 1.3,Gray
5716,800000,Automatic,Foreign Used,58162.0,Nissan,Latio,2015,10,1.2,Silver
8605,1350000,Automatic,Foreign Used,87648.0,Toyota,Sienta,2014,11,1.5 AWD,White
14699,5000000,Automatic,Foreign Used,68000.0,Toyota,Fortuner,2014,11,4.0 RWD,Gold
11015,630000,Automatic,Kenyan Used,150000.0,Toyota,Rav4,2002,23,Automatic,Blue
11709,430000,Automatic,Kenyan Used,NaN,Nissan,Note,2010,15,1.4,Silver


In [16]:
df.isnull().sum()

price              0
transmission       0
condition          0
mileage         2562
make               0
model              0
year               0
car_age            0
engine_info        0
color            190
dtype: int64

In [17]:
df[df['color'].isnull()]

,price,transmission,condition,mileage,make,model,year,car_age,engine_info,color
32,980000,Automatic,Foreign Used,32547.0,Toyota,Ractis,2014,11,1.5,None
135,4300000,Automatic,Foreign Used,36808.0,Lexus,Nx,2015,10,300h,None
178,2900000,Automatic,Foreign Used,70000.0,Subaru,Forester,2014,11,Unknown,None
220,5800000,Automatic,Foreign Used,62415.0,Toyota,Land Cruiser Prado,2014,11,2.7 VVT-i,None
243,2700000,Automatic,Foreign Used,65000.0,Honda,Cr-V,2014,11,EX 4dr SUV (2.4L 4cyl 5A),None
...,...,...,...,...,...,...,...,...,...,...
15284,750000,Automatic,Foreign Used,63000.0,Toyota,Passo,2014,11,1.3,None
15301,750000,Automatic,Foreign Used,63000.0,Toyota,Passo,2014,11,1.3,None
15318,750000,Automatic,Foreign Used,63000.0,Toyota,Passo,2014,11,1.3,None
15334,750000,Automatic,Foreign Used,63000.0,Toyota,Passo,2014,11,1.3,None


In [19]:
df.to_csv('carpredict_eng.csv', index=False, na_rep='NULL')